In [1]:
print("Jai Shree Ram")

Jai Shree Ram


In [19]:
pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 27.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 723.4/723.4 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [openai]2m1/2 [openai]
Note: you may need to restart the kernel to use updated packages.


In [6]:
!pip install PyPDF

In [3]:
from langchain.document_loaders import PyPDFLoader

In [4]:
loader = PyPDFLoader('/Users/anirudh/Downloads/Projects/TCS_RAG_PROJECT/data/10-Q4-2024-As-Filed.pdf')

In [7]:
data = loader.load()

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [9]:
splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=200)

In [10]:
docs = splitter.split_documents(data)

In [11]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser

In [12]:
import os
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

In [16]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small", openai_api_key=OPENAI_API_KEY)

In [17]:
llm = ChatOpenAI(model="gpt-4o", temperature=0, openai_api_key=OPENAI_API_KEY)

/var/folders/b4/5ls6q5d9593br2bzh8lgt8040000gp/T/ipykernel_4317/3869890841.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4o", temperature=0, openai_api_key=OPENAI_API_KEY)


In [20]:
vectorstore = FAISS.from_documents(docs,embeddings)

In [28]:
retriever = vectorstore.as_retriever()

In [30]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser

In [23]:
template="""
    You are an expert Report Analysis bot who analize the report and answer questions.
    Analyze the provided Document to answer related questions.
    Stay relevant to the context, and keep your answers concise and informative.

    CONTEXT:
    {context}

    QUESTION: {question}

    YOUR ANSWER:
    """


In [24]:
from langchain.prompts import ChatPromptTemplate

In [25]:
prompt = ChatPromptTemplate.from_template(template)

In [26]:
from langchain_core.runnables import RunnablePassthrough

In [32]:
parser =StrOutputParser()

In [33]:
rag_chain = (
    {"context": retriever , "question": RunnablePassthrough()}
    | prompt
    | llm
    | parser
)

In [34]:
rag_chain.invoke("What are the Risks that the company is exposed to. Give 5 risks in bullet points.")

"- **Interest Rate Risk**: The company is exposed to fluctuations in U.S. interest rates, which can negatively affect the fair value of its investment portfolio and increase interest expenses on term debt.\n\n- **Foreign Exchange Rate Risk**: Economic risks from foreign exchange rates can impact the company's consolidated financial statements.\n\n- **Component Defects and Product Safety Risks**: Defects in components and products purchased from third parties can make the company's products unsafe, leading to potential environmental or property damage and personal injury.\n\n- **Service Quality and Performance Risks**: The company's service offerings may experience outages, slowdowns, or errors, which can lead to unmet customer expectations and harm the company's reputation.\n\n- **Cybersecurity Risks**: The company faces risks related to unauthorized access to or loss of sensitive data, despite implementing security measures like encryption and authentication technologies."